In [1]:
#install Beautiful Soup and requests for Web Scaping
!pip install BeautifulSoup4
!pip install requests


You should consider upgrading via the 'c:\users\jeevan\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\jeevan\appdata\local\programs\python\python37-32\python.exe -m pip install --upgrade pip' command.


> Getting Data from wikipedia
> parsing html data using BeautifulSoup
> storing the data into Pandas DataFrame

In [2]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#pulling data from wiki page and creating soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#soup = BeautifulSoup(source.text, 'lxml')
soup = BeautifulSoup(source.text, 'html.parser')

#iterating to get the data from htm page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
        
        if(index == 0):
            columns = section
        else:
            data.append(section)

#converting the available data into Pandas DataFrame
toronto_df = pd.DataFrame(data = data, columns = columns)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1A,Not assigned,Not assigned
2,M1A,Not assigned,Not assigned
3,M2A,Not assigned,Not assigned
4,M2A,Not assigned,Not assigned


Removing the 'Not assigned' in Boroughts
More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [3]:
#Remove Boroughts that are 'Not assigned'
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

,Postal Code,Borough,Neighborhood
6,M3A,North York,Parkwoods
7,M3A,North York,Parkwoods
8,M3A,North York,Parkwoods
9,M4A,North York,Victoria Village
10,M4A,North York,Victoria Village


In [4]:
toronto_df["Neighborhood"] = toronto_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ','.join(neigh))

toronto_df = toronto_df.drop_duplicates()

if(toronto_df.index.name != 'Postal Code'):
    toronto_df = toronto_df.set_index('Postal Code')

toronto_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,"Parkwoods,Parkwoods,Parkwoods"
M4A,North York,"Victoria Village,Victoria Village,Victoria Vil..."
M5A,Downtown Toronto,"Regent Park, Harbourfront,Regent Park, Harbour..."
M6A,North York,"Lawrence Manor, Lawrence Heights,Lawrence Mano..."
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,Qu..."


In [5]:
toronto_df['Neighborhood'].replace("Not assigned",toronto_df["Borough"], inplace=True)
toronto_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,"Parkwoods,Parkwoods,Parkwoods"
M4A,North York,"Victoria Village,Victoria Village,Victoria Vil..."
M5A,Downtown Toronto,"Regent Park, Harbourfront,Regent Park, Harbour..."
M6A,North York,"Lawrence Manor, Lawrence Heights,Lawrence Mano..."
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,Qu..."
